In [81]:
import time
start_time = time.time()
import os
import numpy as np
from keras import optimizers
from keras.preprocessing import image
from keras.layers import Dense, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import adam
from keras import models
import cv2
import dlib
import pandas as pd
import matplotlib.pyplot as plt
import sys
import time 
import progressbar
from tqdm.notebook import tqdm_notebook

In [82]:
from sklearn import svm, datasets
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve, ShuffleSplit
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, recall_score, precision_score, f1_score

In [83]:
from tensorflow.python.keras.applications.vgg16 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

In [84]:
df = pd.read_csv('../../dataset/cartoon_set/labels.csv', sep = '\t')
df = df.drop(columns = [df.columns[0]]).drop(columns = [df.columns[1]])
df['face_shape'] = df['face_shape'].apply(str)
df

,face_shape,file_name
0,4,0.png
1,4,1.png
2,3,2.png
3,0,3.png
4,2,4.png
...,...,...
9995,2,9995.png
9996,3,9996.png
9997,2,9997.png
9998,2,9998.png


In [85]:
img = ('../../dataset/cartoon_set/img')
# training, testing = train_test_split(df, random_state=0)
training, validating, testing = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))]) #splitting at n-array
print(training, validating, testing)

     face_shape file_name
5944          2  5944.png
1933          1  1933.png
8701          0  8701.png
7079          3  7079.png
7137          1  7137.png
...         ...       ...
8140          4  8140.png
8997          3  8997.png
6540          1  6540.png
4348          3  4348.png
9114          0  9114.png

[6000 rows x 2 columns]      face_shape file_name
3850          1  3850.png
5130          0  5130.png
8437          0  8437.png
1361          1  1361.png
4551          0  4551.png
...         ...       ...
3808          4  3808.png
2372          0  2372.png
8200          2  8200.png
653           2   653.png
9042          0  9042.png

[2000 rows x 2 columns]      face_shape file_name
4181          2  4181.png
9159          4  9159.png
6974          4  6974.png
3893          3  3893.png
6016          3  6016.png
...         ...       ...
9409          2  9409.png
4499          3  4499.png
3825          0  3825.png
3973          0  3973.png
142           1   142.png

[2000 rows x 

In [86]:
# set up data generator
data_generator = ImageDataGenerator(
    rescale = 1./255.,
#     validation_split = 0.25,
    horizontal_flip=True,
    vertical_flip=True   
)

# # get batches of training images from the df
# train_generator = data_generator.flow_from_dataframe(
#         dataframe=train_set,
#         x_col="file_name",
#         y_col="face_shape",
#         target_size=(178, 218),
#         batch_size=12,
#         class_mode='categorical')

# validate_generator = data_generator.flow_from_dataframe(
#         dataframe=train_set,
#         x_col="file_name",
#         y_col="face_shape",
#         target_size=(178, 218),
#         batch_size=12,
#         class_mode='categorical')

# Get batches of training dataset from the dataframe
print("Training Dataset Preparation: ")
train_generator = data_generator.flow_from_dataframe(
        dataframe = training, directory = img ,
        x_col = "file_name", y_col = "face_shape",
        class_mode = 'categorical', target_size = (30,30),
        batch_size = 32, subset = 'training') 
   
# Get batches of validation dataset from the dataframe
print("\nValidation Dataset Preparation: ")
validation_generator = data_generator.flow_from_dataframe(
        dataframe = validating, directory = img ,
        x_col = "file_name", y_col = "face_shape",
        class_mode = 'categorical', target_size = (30,30),
        batch_size = 32, subset = 'training')

Training Dataset Preparation: 
Found 6000 validated image filenames belonging to 5 classes.

Validation Dataset Preparation: 
Found 2000 validated image filenames belonging to 5 classes.


In [87]:
# starting point 
my_model= models.Sequential()

# Add first convolutional block
my_model.add(Conv2D(16, (3, 3), activation='relu', padding='same', 
                    input_shape=(30,30,3))) #height, width, depth
my_model.add(MaxPooling2D((2, 2), padding='same'))
# second block
my_model.add(Conv2D(32, (3, 3), activation='relu', padding='same')) #Convolution: filters, kernel_size that specifies the height and width of the 2D convolution window, p padding layers so dimensions of input = output
my_model.add(MaxPooling2D((2, 2), padding='same'))
# third block
my_model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
my_model.add(MaxPooling2D((2, 2), padding='same'))
# fourth block
my_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
my_model.add(MaxPooling2D((2, 2), padding='same'))
# global average pooling
#my_model.add(GlobalAveragePooling2D())
# fully connected layer
#my_model.add(Dense(64, activation='relu'))
#my_model.add(BatchNormalization())
# make predictions
my_model.add(Flatten())
my_model.add(Dense(5, activation='softmax'))
# Show a summary of the model. Check the number of trainable parameters
my_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 30, 30, 16)        448       
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 15, 15, 16)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 15, 15, 32)        4640      
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 8, 8, 64)          18496     
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 4, 4, 64)          0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 4, 4, 128)        

In [88]:
# # use early stopping to optimally terminate training through callbacks

# es=EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# # save best model automatically
# mc= ModelCheckpoint('/../../dataset/cartoon_set', monitor='val_loss', 
#                     mode='min', verbose=1, save_best_only=True)
# cb_list=[es,mc]


# compile model 
my_model.compile(optimizer='adam', loss='categorical_crossentropy', 
                 metrics=['accuracy'])


In [90]:
batch_size = 32
history = my_model.fit_generator(
                                train_generator,
                                epochs=25,
                                steps_per_epoch=train_generator.samples // batch_size,
                                validation_data=validation_generator,
                                validation_steps=validation_generator.samples // batch_size
                                )

Epoch 1/25
187/187 [==============================] - 40s 216ms/step - loss: 1.5229 - accuracy: 0.2656 - val_loss: 1.2223 - val_accuracy: 0.3765
Epoch 2/25
187/187 [==============================] - 28s 152ms/step - loss: 1.1513 - accuracy: 0.4358 - val_loss: 1.1478 - val_accuracy: 0.5412
Epoch 3/25
187/187 [==============================] - 29s 156ms/step - loss: 0.9639 - accuracy: 0.5736 - val_loss: 0.7487 - val_accuracy: 0.6956
Epoch 4/25
187/187 [==============================] - 29s 154ms/step - loss: 0.6446 - accuracy: 0.7348 - val_loss: 0.6485 - val_accuracy: 0.6911
Epoch 5/25
187/187 [==============================] - 30s 160ms/step - loss: 0.5227 - accuracy: 0.7864 - val_loss: 0.4566 - val_accuracy: 0.7795
Epoch 6/25
187/187 [==============================] - 30s 163ms/step - loss: 0.4335 - accuracy: 0.8266 - val_loss: 0.2843 - val_accuracy: 0.8308
Epoch 7/25
187/187 [==============================] - 30s 158ms/step - loss: 0.3904 - accuracy: 0.8430 - val_loss: 0.4346 - val_ac

In [65]:
test_generator = data_generator.flow_from_dataframe(
        dataframe = testing, directory = img,
        x_col = "file_name", y_col = "face_shape",
        batch_size=32,
        class_mode='categorical', target_size=(30, 30),
        shuffle=False)

Found 2000 validated image filenames belonging to 5 classes.


In [91]:
test_generator.reset()
pred=saved_model.predict_generator(test_generator, verbose=1, steps=1000)
# determine the maximum activation value for each sample
predicted_class_indices=np.argmax(pred,axis=1)

# label each predicted value to correct gender
labels = (test_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

# format file names to simply male or female
filenames=test_generator.filenames
filenz=[0]
for i in range(0,len(filenames)):
    filenz.append(filenames[i].split('\\')[0])
filenz=filenz[1:]

# determine the test set accuracy
match=[]
for i in range(0,len(filenames)):
    match.append(filenz[i]==predictions[i])
match.count(True)/1000



NameError: name 'saved_model' is not defined

In [92]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 10919.487528562546 seconds ---
